<a href="https://colab.research.google.com/github/MHMotamedi97/Forex_Trader/blob/main/Data_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this version of Data Processing module we read data from a csv file.
we get csv file with a following code from mt5 server.
Also this version is only for one asset.

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [18]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=57b8aac9b4270a4df4ef237f704c076ccb9c6ae6b6fb7b1c0bdd0f9cfd3b69a9
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
#@title getting data from mt5 server

import MetaTrader5 as mt5
from datetime import datetime, timezone

def get_data(symbol, timeframe, date_from, date_to):
    data = mt5.copy_rates_range(symbol, timeframe, date_from, date_to)
    data = pd.DataFrame(data)
    data = data[['time', 'open', 'high', 'low', 'close']]
    data.columns=['Time', 'Open', 'High', 'Low', 'Close']
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    return data

if __name__=='__main__':
  mt5.initialize()
  symbol = 'EURUSD'
  timeframe = mt5.TIMEFRAME_M1
  date_from = datetime(2018, 1, 1, 00, 00, tzinfo=timezone.utc)
  date_to = datetime(2023, 1, 1, 0, 0, tzinfo=timezone.utc)
  # # date_to = datetime.now(tz=timezone.utc)
  data = get_data(symbol, timeframe, date_from, date_to)
  mt5.shutdown()


In [11]:
#@title function for changing timeframe
def change_timeframe(dataset, datetime='time', change_to="1H"):
    """
    Info:
    :param dataset: dataset has time stamp and need to chage datetime
    :param timestamp: default is Timestamp
    :param change_to: change duration to ..., default : H


        # H      Hour (24-hour clock) as a decimal number [00,23].
        # d      Day of the month as a decimal number [01,31].
        # m       Month as a decimal number [01,12].
        # M       Minute as a decimal number [00,59]. 30min
        # Y       Year with century as a decimal number.
    :return:
    """
    dataset['timestamp'] = dataset[datetime].view(int) // 10 ** 9
    dataset =dataset.sort_values(by='timestamp', ascending=True)
    change_to_datetime = pd.to_datetime(dataset['timestamp'], unit='s')
    change_period = pd.Series(change_to_datetime).dt.floor(change_to)
    dataset[datetime] = change_period 

    data = dataset.groupby([datetime]).sum().reset_index().round(8)  

    data[datetime] = dataset.groupby(
        [datetime]).min().reset_index().round(8)[datetime]
    if 'high' in dataset.columns:
        data['high'] = dataset.groupby(
            [datetime]).max().reset_index().round(8)['high']
    if 'low' in dataset.columns:
        data['low'] = dataset.groupby(
            [datetime]).min().reset_index().round(8)['low']
    if 'open' in dataset.columns:
        data['open'] = dataset.groupby(
            [datetime]).first().reset_index().round(8)['open']
    if 'close' in dataset.columns:
        data['close'] = dataset.groupby(
            [datetime]).last().reset_index().round(8)['close']

    del dataset
    # data['time_scale'] = change_to
    data = data.drop(columns=['timestamp'])
    return data


In [78]:
#@title loading data from drive
import pandas as pd

downloaded = drive.CreateFile({'id':'18sCPfC7gJfvIhSvf_2FbmsLXtzl7r3Ty'}) # replace the id with id of file you want to access
downloaded.GetContentFile('EURUSD.csv')

data = pd.read_csv('EURUSD.csv')
data['time'] = pd.to_datetime(data['time'])
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2010-01-04 00:00:00,1.43259,1.43259,1.43213,1.43222,28,18,0
1,2010-01-04 00:01:00,1.43227,1.43242,1.43223,1.43241,12,18,0
2,2010-01-04 00:02:00,1.43234,1.43244,1.43231,1.43235,22,18,0
3,2010-01-04 00:03:00,1.43236,1.43237,1.43231,1.43231,5,18,0
4,2010-01-04 00:04:00,1.43232,1.43235,1.43225,1.43235,14,18,0
...,...,...,...,...,...,...,...,...
4446095,2021-12-31 20:55:00,1.13842,1.13844,1.13842,1.13842,8,7,0
4446096,2021-12-31 20:56:00,1.13846,1.13846,1.13844,1.13844,7,7,0
4446097,2021-12-31 20:57:00,1.13844,1.13847,1.13841,1.13847,21,7,0
4446098,2021-12-31 20:58:00,1.13847,1.13847,1.13838,1.13839,10,7,0


### Cleaning and Preprocessing data


In [79]:
data = change_timeframe(data, change_to='1H')
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2010-01-04 00:00:00,1.43259,1.43336,1.43151,1.43153,969,989,0
1,2010-01-04 01:00:00,1.43151,1.43153,1.42879,1.42886,2098,918,0
2,2010-01-04 02:00:00,1.42885,1.42885,1.42569,1.42705,2082,974,0
3,2010-01-04 03:00:00,1.42702,1.42989,1.42700,1.42939,1544,963,0
4,2010-01-04 04:00:00,1.42938,1.42968,1.42718,1.42848,1131,971,0
...,...,...,...,...,...,...,...,...
74390,2021-12-31 16:00:00,1.13476,1.13526,1.13378,1.13435,3723,62,0
74391,2021-12-31 17:00:00,1.13436,1.13752,1.13376,1.13717,5439,103,0
74392,2021-12-31 18:00:00,1.13717,1.13793,1.13588,1.13787,3783,56,0
74393,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860,1743,132,0


In [80]:
data = data[['time', 'open', 'high', 'low', 'close']]
data['time'] = pd.to_datetime(data['time'])
data['day'] = data['time'].dt.dayofweek
data

<ipython-input-80-3c2d6840441e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time'] = pd.to_datetime(data['time'])
<ipython-input-80-3c2d6840441e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = data['time'].dt.dayofweek


,time,open,high,low,close,day
0,2010-01-04 00:00:00,1.43259,1.43336,1.43151,1.43153,0
1,2010-01-04 01:00:00,1.43151,1.43153,1.42879,1.42886,0
2,2010-01-04 02:00:00,1.42885,1.42885,1.42569,1.42705,0
3,2010-01-04 03:00:00,1.42702,1.42989,1.42700,1.42939,0
4,2010-01-04 04:00:00,1.42938,1.42968,1.42718,1.42848,0
...,...,...,...,...,...,...
74390,2021-12-31 16:00:00,1.13476,1.13526,1.13378,1.13435,4
74391,2021-12-31 17:00:00,1.13436,1.13752,1.13376,1.13717,4
74392,2021-12-31 18:00:00,1.13717,1.13793,1.13588,1.13787,4
74393,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860,4


In [81]:
# adding indicators 
# here I add three indicators, 
import pandas_ta
data.ta.zscore(close=data.close, length=24, append=True)
data.ta.rsi(close=data.close, length=14, append=True)
data.ta.sma(close=data.close, length=12, append=True)
data = data.dropna().reset_index(drop=True)
data

<ipython-input-81-02b3aae4310b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ta.zscore(close=data.close, length=24, append=True)


,time,open,high,low,close,day,ZS_24,RSI_14,SMA_12
0,2010-01-04 23:00:00,1.44111,1.44216,1.44089,1.44111,0,0.813686,66.812473,1.441397
1,2010-01-05 00:00:00,1.44107,1.44254,1.44056,1.44240,1,0.950791,70.333890,1.441662
2,2010-01-05 01:00:00,1.44241,1.44391,1.44120,1.44183,1,0.784827,66.953360,1.441835
3,2010-01-05 02:00:00,1.44180,1.44322,1.44143,1.44189,1,0.734517,67.132441,1.441784
4,2010-01-05 03:00:00,1.44188,1.44188,1.44058,1.44142,1,0.584128,64.197684,1.441681
...,...,...,...,...,...,...,...,...,...
74367,2021-12-31 16:00:00,1.13476,1.13526,1.13378,1.13435,4,2.216139,59.001947,1.132268
74368,2021-12-31 17:00:00,1.13436,1.13752,1.13376,1.13717,4,3.424004,67.516676,1.132734
74369,2021-12-31 18:00:00,1.13717,1.13793,1.13588,1.13787,4,2.933610,69.225258,1.133244
74370,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860,4,2.646098,70.941796,1.133870


In [82]:
# scaling data
# Assuming you have a time series stored in a pandas DataFrame called 'data'
window_size = 10  # Define the size of the rolling window
# Calculate the rolling mean and standard deviation
rolling_mean = data.drop(columns=['time', 'day']).rolling(window=window_size).mean()
rolling_std = data.drop(columns=['time', 'day']).rolling(window=window_size).std()
# Normalize the data using the rolling mean and standard deviation
normalized_data = (data.drop(columns=['time', 'day']) - rolling_mean) / rolling_std
# Optional: Scale the data to a specific range, such as between 0 and 1
min_value = normalized_data.min()
max_value = normalized_data.max()
scaled_data = (normalized_data - min_value) / (max_value - min_value)
data[['n_open', 'n_high', 'n_low', 'n_close', 'ZS_24', 'RSI_14', 'SMA_12']] = scaled_data
data = data.dropna().reset_index(drop=True)
data

,time,open,high,low,close,day,ZS_24,RSI_14,SMA_12,n_open,n_high,n_low,n_close
0,2010-01-05 08:00:00,1.44749,1.44830,1.44472,1.44520,1,0.627820,0.410213,0.874502,0.870570,0.784547,0.828155,0.655235
1,2010-01-05 09:00:00,1.44521,1.44537,1.44042,1.44162,1,0.176741,0.129360,0.793225,0.655396,0.551913,0.344056,0.370758
2,2010-01-05 10:00:00,1.44161,1.44372,1.44141,1.44312,1,0.390866,0.284254,0.764159,0.367931,0.414348,0.443398,0.483768
3,2010-01-05 11:00:00,1.44316,1.44370,1.44134,1.44177,1,0.239601,0.253652,0.722077,0.485297,0.414823,0.433387,0.376584
4,2010-01-05 12:00:00,1.44176,1.44249,1.43995,1.44176,1,0.273317,0.299521,0.669522,0.373334,0.330499,0.304434,0.377369
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74358,2021-12-31 16:00:00,1.13476,1.13526,1.13378,1.13435,4,0.737742,0.738949,0.910070,0.900359,0.765430,0.910439,0.759005
74359,2021-12-31 17:00:00,1.13436,1.13752,1.13376,1.13717,4,0.796225,0.844549,0.937537,0.758051,0.856541,0.794174,0.881990
74360,2021-12-31 18:00:00,1.13717,1.13793,1.13588,1.13787,4,0.693252,0.788092,0.903386,0.879279,0.796248,0.870006,0.814534
74361,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860,4,0.635571,0.768759,0.877877,0.812901,0.775774,0.860028,0.786348


In [83]:
TRAIN_END_DATE = '2020-01-01'
train = data[data.time < TRAIN_END_DATE]
test  = data[data.time >= TRAIN_END_DATE]


In [84]:
train

,time,open,high,low,close,day,ZS_24,RSI_14,SMA_12,n_open,n_high,n_low,n_close
0,2010-01-05 08:00:00,1.44749,1.44830,1.44472,1.44520,1,0.627820,0.410213,0.874502,0.870570,0.784547,0.828155,0.655235
1,2010-01-05 09:00:00,1.44521,1.44537,1.44042,1.44162,1,0.176741,0.129360,0.793225,0.655396,0.551913,0.344056,0.370758
2,2010-01-05 10:00:00,1.44161,1.44372,1.44141,1.44312,1,0.390866,0.284254,0.764159,0.367931,0.414348,0.443398,0.483768
3,2010-01-05 11:00:00,1.44316,1.44370,1.44134,1.44177,1,0.239601,0.253652,0.722077,0.485297,0.414823,0.433387,0.376584
4,2010-01-05 12:00:00,1.44176,1.44249,1.43995,1.44176,1,0.273317,0.299521,0.669522,0.373334,0.330499,0.304434,0.377369
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61903,2019-12-31 16:00:00,1.12380,1.12389,1.12178,1.12197,1,0.374288,0.339108,0.774901,0.818759,0.749500,0.609553,0.535846
61904,2019-12-31 17:00:00,1.12197,1.12324,1.12169,1.12235,1,0.415897,0.388503,0.764173,0.530912,0.630566,0.571062,0.572662
61905,2019-12-31 18:00:00,1.12230,1.12309,1.12159,1.12261,1,0.427161,0.411345,0.762466,0.560253,0.583184,0.532125,0.589794
61906,2019-12-31 19:00:00,1.12261,1.12291,1.12225,1.12288,1,0.416463,0.429840,0.761537,0.585655,0.524410,0.631567,0.614600


In [85]:
test

,time,open,high,low,close,day,ZS_24,RSI_14,SMA_12,n_open,n_high,n_low,n_close
61908,2020-01-02 06:00:00,1.12130,1.12141,1.12006,1.12009,3,0.110822,0.127201,0.706197,0.262933,0.167109,0.142976,0.095126
61909,2020-01-02 07:00:00,1.12009,1.12041,1.12006,1.12041,3,0.233721,0.245190,0.657964,0.133304,0.113809,0.202809,0.222805
61910,2020-01-02 08:00:00,1.12041,1.12074,1.12035,1.12071,3,0.304702,0.319057,0.612610,0.244554,0.243480,0.297438,0.312574
61911,2020-01-02 09:00:00,1.12075,1.12100,1.12041,1.12096,3,0.357563,0.374763,0.553009,0.331720,0.320795,0.341937,0.377082
61912,2020-01-02 10:00:00,1.12096,1.12133,1.12050,1.12116,3,0.401102,0.423175,0.425152,0.388391,0.391859,0.383444,0.432934
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74358,2021-12-31 16:00:00,1.13476,1.13526,1.13378,1.13435,4,0.737742,0.738949,0.910070,0.900359,0.765430,0.910439,0.759005
74359,2021-12-31 17:00:00,1.13436,1.13752,1.13376,1.13717,4,0.796225,0.844549,0.937537,0.758051,0.856541,0.794174,0.881990
74360,2021-12-31 18:00:00,1.13717,1.13793,1.13588,1.13787,4,0.693252,0.788092,0.903386,0.879279,0.796248,0.870006,0.814534
74361,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860,4,0.635571,0.768759,0.877877,0.812901,0.775774,0.860028,0.786348


In [87]:
#@title Saving data for further use
path = '/content/drive/MyDrive/RL_Trading'
train.to_csv(path+'/train_data.csv', index=None)
test.to_csv(path+'/test_data.csv', index=None)
